In [1]:
import geopandas as gp
import pandas as pd
import numpy as np
import jupyter_beeper

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_seq_items = None

In [3]:
parks = gp.read_file('../Data/Parks.geojson')
house = pd.read_csv('../Data/Cleaned/House/house_with_neigh.csv')

In [4]:
parks.head(1)

,OBJECTID,GEO_ID,TYPE_DESC,SCODE_NAME,LCODE_NAME,NAME,geometry
0,180,180,CGSP,None,3017,CITY WIDE OPEN SPACE,"POLYGON ((-79.22672 43.80698, -79.22690 43.806..."


In [5]:
house.head(1)

,lat,long,sqft,parking,mean_district_income,bedrooms_bg,bedrooms_ag,bathrooms,final_price,type_Att/Row/Twnhouse,type_Co-Op Apt,type_Co-Ownership Apt,type_Comm Element Condo,type_Condo Apt,type_Condo Townhouse,type_Detached,type_Link,type_Plex,type_Semi-Detached,type_Store W/Apt/Offc,neighbourhood,geometry
0,43.753182,-79.507053,NaN,4,29958,1,3,2,930000,0,0,0,0,0,0,1,0,0,0,0,York University Heights,"MULTIPOLYGON (((-79.50528799999999 43.759873, ..."


Taking centroid of park for easy calculations

In [6]:
parks['centroid'] = parks.to_crs("EPSG:4326").centroid

<ipython-input-6-a8f2e04e3b32>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parks['centroid'] = parks.to_crs("EPSG:4326").centroid


In [7]:
parks.head(2)

,OBJECTID,GEO_ID,TYPE_DESC,SCODE_NAME,LCODE_NAME,NAME,geometry,centroid
0,180,180,CGSP,None,3017,CITY WIDE OPEN SPACE,"POLYGON ((-79.22672 43.80698, -79.22690 43.806...",POINT (-79.22702 43.80633)
1,193,193,CGSP,None,3050,PUBLIC ACCESS PROPERTY,"POLYGON ((-79.20123 43.80354, -79.20121 43.803...",POINT (-79.20143 43.80327)


In [8]:
parks.centroid.head(1)

<ipython-input-8-a6c8a764a72b>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parks.centroid.head(1)


0    POINT (-79.22702 43.80633)
dtype: geometry

In [9]:
gpf_house = gp.GeoDataFrame(house, crs="EPSG:4326", geometry=gp.points_from_xy(house.long,house.lat))

In [10]:
parks.shape

(3274, 8)

In [11]:
parks[parks.geometry.isna()]

,OBJECTID,GEO_ID,TYPE_DESC,SCODE_NAME,LCODE_NAME,NAME,geometry,centroid
1536,4310522,473,CGSP,473,473,HEATHER HEIGHTS WOODS,None,None


In [12]:
parks.drop(1536, axis=0,inplace=True)

In [13]:
parks.shape

(3273, 8)

In [14]:
gpf_house.head(1)

,lat,long,sqft,parking,mean_district_income,bedrooms_bg,bedrooms_ag,bathrooms,final_price,type_Att/Row/Twnhouse,type_Co-Op Apt,type_Co-Ownership Apt,type_Comm Element Condo,type_Condo Apt,type_Condo Townhouse,type_Detached,type_Link,type_Plex,type_Semi-Detached,type_Store W/Apt/Offc,neighbourhood,geometry
0,43.753182,-79.507053,NaN,4,29958,1,3,2,930000,0,0,0,0,0,0,1,0,0,0,0,York University Heights,POINT (-79.50705 43.75318)


Check nearest parks distance

In [15]:
from scipy.spatial import cKDTree

def ckdnearest(gdA, gdB, distcol):

    nA = np.array(list(gdA.geometry.to_crs('EPSG:5234').apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.centroid.to_crs('EPSG:5234').apply(lambda x: (x.x, x.y))))
    
    btree = cKDTree(nB)
    
    dist, idx = btree.query(nA, k=1)
    #If you want other features from B
    #gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            #If you want other features from B
            #gdB_nearest,
            pd.Series(dist, name=distcol)
        ], 
        axis=1)
    
    gdf[distcol] = gdf[distcol].apply(lambda x : round(x,2))

    return gdf

In [16]:
gpf_house = ckdnearest(gpf_house, parks, 'dist_nearest_park' )

<ipython-input-15-e594dc693f2a>:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nB = np.array(list(gdB.centroid.to_crs('EPSG:5234').apply(lambda x: (x.x, x.y))))


In [17]:
gpf_house.sort_values(by='dist_nearest_park',ascending=False ).head()

,lat,long,sqft,parking,mean_district_income,bedrooms_bg,bedrooms_ag,bathrooms,final_price,type_Att/Row/Twnhouse,type_Co-Op Apt,type_Co-Ownership Apt,type_Comm Element Condo,type_Condo Apt,type_Condo Townhouse,type_Detached,type_Link,type_Plex,type_Semi-Detached,type_Store W/Apt/Offc,neighbourhood,geometry,dist_nearest_park
15097,43.715040,-79.365154,NaN,2,125564,1,3,4,1850000,0,0,0,0,0,0,1,0,0,0,0,Leaside-Bennington,POINT (-79.36515 43.71504),822.36
10789,43.755657,-79.449182,2750.0,4,45936,1,4,4,1725000,0,0,0,0,0,0,1,0,0,0,0,Bathurst Manor,POINT (-79.44918 43.75566),813.92
15100,43.715615,-79.365234,2250.0,2,125564,1,4,4,2388000,0,0,0,0,0,0,1,0,0,0,0,Leaside-Bennington,POINT (-79.36523 43.71561),797.34
10778,43.753577,-79.449688,1100.0,1,45936,0,2,2,555000,0,0,0,0,1,0,0,0,0,0,0,Bathurst Manor,POINT (-79.44969 43.75358),788.47
15096,43.714878,-79.364090,NaN,4,125564,1,2,2,1420000,0,0,0,0,0,0,1,0,0,0,0,Leaside-Bennington,POINT (-79.36409 43.71488),757.61


In [22]:
gpf_house.to_csv('../Data/Cleaned/House/house_with_park.csv', index=False)

In [23]:
heh = pd.read_csv('../Data/Cleaned/House/house_with_park.csv')

In [24]:
heh.head()

,lat,long,sqft,parking,mean_district_income,bedrooms_bg,bedrooms_ag,bathrooms,final_price,type_Att/Row/Twnhouse,type_Co-Op Apt,type_Co-Ownership Apt,type_Comm Element Condo,type_Condo Apt,type_Condo Townhouse,type_Detached,type_Link,type_Plex,type_Semi-Detached,type_Store W/Apt/Offc,neighbourhood,geometry,dist_nearest_park
0,43.753182,-79.507053,NaN,4,29958,1,3,2,930000,0,0,0,0,0,0,1,0,0,0,0,York University Heights,POINT (-79.50705339999998 43.75318170000001),130.27
1,43.753090,-79.491536,1300.0,1,29958,0,3,3,413000,0,0,0,0,0,1,0,0,0,0,0,York University Heights,POINT (-79.49153570000001 43.75309),134.63
2,43.753090,-79.491536,1300.0,3,29958,1,3,2,400000,0,0,0,0,0,1,0,0,0,0,0,York University Heights,POINT (-79.49153570000001 43.75309),134.63
3,43.753177,-79.490821,950.0,1,29958,0,2,1,343000,0,0,0,0,1,0,0,0,0,0,0,York University Heights,POINT (-79.4908207 43.7531774),149.17
4,43.753177,-79.490821,650.0,1,29958,0,1,1,318000,0,0,0,0,1,0,0,0,0,0,0,York University Heights,POINT (-79.4908207 43.7531774),149.17
